<a href="https://colab.research.google.com/github/ApurvaMayank-iitb/24D0894_IE643_NIFTI/blob/main/Auto_Encoder_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from google.colab import files
import torch

# Step 1: Upload the model file
uploaded = files.upload()  # This will prompt you to upload `conv_autoencoder_model.pth`

# Step 2: Verify the model path
model_path = "/content/conv_autoencoder_model.pth"  # Colab saves uploads in the `/content` directory

# Check if the model has been uploaded
if model_path in uploaded:
    print("Model uploaded successfully.")
else:
    print("Model upload failed. Please try again.")


Saving conv_autoencoder_model.pth to conv_autoencoder_model.pth
Model upload failed. Please try again.


In [ ]:
from google.colab import files
import torch

# Upload the model file
uploaded = files.upload()  # Upload `conv_autoencoder_model.pth`

# Get the uploaded file path (it will be stored in `/content` by default)
model_path = list(uploaded.keys())[0]
print(f"Model uploaded successfully and saved as {model_path} in Colab.")


Saving Generator_trained.pth to Generator_trained.pth
Model uploaded successfully and saved as Generator_trained.pth in Colab.


In [ ]:
# Save Gradio interface code to `app.py`
interface_code = f"""
import gradio as gr
import torch
import nibabel as nib
import numpy as np
from model import ConvAutoencoder  # Ensure ConvAutoencoder class is available

# Load the pre-trained model from the uploaded file path
model_path = "/content/{model_path}"  # Full path of the uploaded model file
model = ConvAutoencoder()
model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
model.eval()

# Prediction function
def predict(modalities):
    slices = [nib.load(modality.name).get_fdata() for modality in modalities]
    slice_avg = np.mean(slices, axis=0)  # Combine modalities by averaging
    normalized_slice = (slice_avg - slice_avg.min()) / (slice_avg.max() - slice_avg.min())  # Normalize
    tensor_slice = torch.tensor(normalized_slice).unsqueeze(0).unsqueeze(0).float()  # Convert to tensor with batch and channel

    with torch.no_grad():
        reconstruction = model(tensor_slice)
    reconstruction_error = torch.abs(reconstruction - tensor_slice).squeeze().numpy()
    return reconstruction_error  # Return error map for visualization

# Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.inputs.File(type="file", label="Upload NIfTI Modalities (e.g., T1, T2, FLAIR)", multiple=True),
    outputs="image",
    title="Brain MRI Anomaly Detection - ConvAutoencoder",
    description="Upload NIfTI files (e.g., T1, T2, FLAIR) for brain anomaly detection using a ConvAutoencoder.",
)

iface.launch()
"""

# Write the code to `app.py`
with open("app.py", "w") as file:
    file.write(interface_code)


In [ ]:
# Create requirements.txt with necessary dependencies
requirements = """
gradio
torch
nibabel
numpy
"""
with open("requirements.txt", "w") as file:
    file.write(requirements)


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import Repository

# Set up repository details
repo_url = "https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface"  # Update with your actual space URL

# Initialize the repository
repo = Repository(local_dir=".", clone_from=repo_url)

# Commit and push all files to the Space repository
repo.push_to_hub(commit_message="Initial commit with Gradio ConvAutoencoder interface")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  

OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('/content/.'). If you really want to do this, do it manually:
 cd /content/. && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [ ]:
from huggingface_hub import Repository

# Create a new directory for the repository
repo_dir = "/content/huggingface_space_repo"  # Define a clean directory for the repository

# Initialize the repository in this new directory
repo_url = "https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface"  # Replace with your space URL
repo = Repository(local_dir=repo_dir, clone_from=repo_url)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface into local empty directory.


In [ ]:
import shutil

# Move the necessary files to the repo directory
shutil.move("/content/app.py", repo_dir + "/app.py")
shutil.move("/content/requirements.txt", repo_dir + "/requirements.txt")
shutil.move("/content/conv_autoencoder_model.pth", repo_dir + "/conv_autoencoder_model.pth")


'/content/huggingface_space_repo/conv_autoencoder_model.pth'

In [ ]:
# Change directory to the repository directory and push
repo.push_to_hub(commit_message="Initial commit with Gradio ConvAutoencoder interface")


OSError: Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@36b291f0e417.(none)')


In [ ]:
# Configure Git with your Hugging Face username and email
!git config --global user.email "24D0894@iitb.ac.in"
!git config --global user.name "24D0894"


In [ ]:
# Push the changes to Hugging Face Spaces
repo.push_to_hub(commit_message="Initial commit with Gradio ConvAutoencoder interface")


batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface'

error: failed to push some refs to 'https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface'



OSError: batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface'


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
repo.push_to_hub(commit_message="Initial commit with Gradio ConvAutoencoder interface")


In [ ]:
%cd /content/huggingface_space_repo


/content/huggingface_space_repo


In [ ]:
from huggingface_hub import Repository
repo = Repository(local_dir=".", clone_from="https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface")
repo.push_to_hub(commit_message="Re-pushing files to Space")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/huggingface_space_repo/. is already a clone of https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
# Pull latest changes to make sure everything is up-to-date
repo.git_pull()


In [ ]:
# List files in the repository directory to confirm they are present
!ls /content/huggingface_space_repo


app.py	conv_autoencoder_model.pth  README.md  requirements.txt


In [ ]:
# Push all files to Hugging Face Space
repo.push_to_hub(commit_message="Final commit with all required files")


In [ ]:
!ls /content/huggingface_space_repo


app.py	conv_autoencoder_model.pth  README.md  requirements.txt


In [ ]:
%cd /content/huggingface_space_repo


/content/huggingface_space_repo


In [ ]:
!git status


On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [ ]:
!git push origin main


batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface'


In [ ]:
%cd /content


/content


In [ ]:
!rm -rf huggingface_space_repo


In [ ]:
from huggingface_hub import Repository
repo_url = "https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface"
repo = Repository(local_dir="huggingface_space_repo", clone_from=repo_url)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface into local empty directory.


In [ ]:
import shutil

# Copy files into the repository directory
shutil.copy("/content/app.py", "/content/huggingface_space_repo/app.py")
shutil.copy("/content/requirements.txt", "/content/huggingface_space_repo/requirements.txt")
shutil.copy("/content/conv_autoencoder_model.pth", "/content/huggingface_space_repo/conv_autoencoder_model.pth")


FileNotFoundError: [Errno 2] No such file or directory: '/content/app.py'

In [ ]:
!ls /content


Generator_trained.pth  huggingface_space_repo  sample_data


In [ ]:
# Generate app.py with Gradio interface code
with open("/content/app.py", "w") as f:
    f.write("""<YOUR APP.PY CODE HERE>""")

# Create requirements.txt
with open("/content/requirements.txt", "w") as f:
    f.write("gradio\ntorch\nnibabel\nnumpy\n")


In [ ]:
import shutil

# Copy files into the repository directory
shutil.copy("/content/app.py", "/content/huggingface_space_repo/app.py")
shutil.copy("/content/requirements.txt", "/content/huggingface_space_repo/requirements.txt")
shutil.copy("/content/conv_autoencoder_model.pth", "/content/huggingface_space_repo/conv_autoencoder_model.pth")


FileNotFoundError: [Errno 2] No such file or directory: '/content/conv_autoencoder_model.pth'

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload `conv_autoencoder_model.pth` manually


Saving conv_autoencoder_model.pth to conv_autoencoder_model.pth


In [ ]:
import shutil

# Copy files into the repository directory
shutil.copy("/content/app.py", "/content/huggingface_space_repo/app.py")
shutil.copy("/content/requirements.txt", "/content/huggingface_space_repo/requirements.txt")
shutil.copy("/content/conv_autoencoder_model.pth", "/content/huggingface_space_repo/conv_autoencoder_model.pth")


'/content/huggingface_space_repo/conv_autoencoder_model.pth'

In [ ]:
%cd /content/huggingface_space_repo
!git add app.py requirements.txt conv_autoencoder_model.pth
!git commit -m "Add application files for ConvAutoencoder Gradio interface"
!git push origin main


/content/huggingface_space_repo
[main 6d38478] Add application files for ConvAutoencoder Gradio interface
 3 files changed, 8 insertions(+)
 create mode 100644 app.py
 create mode 100644 conv_autoencoder_model.pth
 create mode 100644 requirements.txt
batch response: Authorization error.
Uploading LFS objects:   0% (0/1), 0 B | 0 B/s, done.
error: failed to push some refs to 'https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface'


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
%cd /content/huggingface_space_repo


/content/huggingface_space_repo


In [ ]:
!git lfs install
!git add app.py requirements.txt conv_autoencoder_model.pth
!git commit -m "Add application files for ConvAutoencoder Gradio interface"
!git push origin main


Updated git hooks.
Git LFS initialized.
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/spaces/24D0894/Auto_Encoder_Interface'


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import upload_file

# Define the repository details
repo_id = "24D0894/Auto_Encoder_Interface"

# Upload each file to the Space repository
upload_file(
    path_or_fileobj="/content/huggingface_space_repo/app.py",
    path_in_repo="app.py",
    repo_id=repo_id,
    repo_type="space"
)

upload_file(
    path_or_fileobj="/content/huggingface_space_repo/requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=repo_id,
    repo_type="space"
)

upload_file(
    path_or_fileobj="/content/huggingface_space_repo/conv_autoencoder_model.pth",
    path_in_repo="conv_autoencoder_model.pth",
    repo_id=repo_id,
    repo_type="space"
)


HfHubHTTPError: (Request ID: Root=1-67262963-4b3e82a470275f401791051c;3e117e84-de3c-4ec7-9e79-b5eda2754104)

403 Forbidden: Forbidden: pass `create_pr=1` as a query parameter to create a Pull Request.
Cannot access content at: https://huggingface.co/api/spaces/24D0894/Auto_Encoder_Interface/commit/main.
Make sure your token has the correct permissions.